In [ ]:
import os
import cv2
import numpy as np
from scipy.signal import convolve2d

BASE_PATH = 'MMU-Iris-Database'
TEMPLATE_DIR = 'templates'
os.makedirs(TEMPLATE_DIR, exist_ok=True)

REG_SUBJ, REG_EYE, REG_SHOT = 1, 'left', 'aeval1.bmp'  # 등록할 홍채 이미지
SAME_SHOT = 'bryanl1.bmp'  # 동일 피험자 다른 촬영 이미지
DIFF_SUBJ = 5

# 임계값
MAX_SHIFT = 32

In [ ]:
# 홍채 인식 실습 - 등록

def img_path(subj: int, eye: str, shot: str) -> str:
    return os.path.join(BASE_PATH, str(subj), eye, shot)

def load_iris(subj: int, eye: str, shot: str):
    path = img_path(subj, eye, shot)
    if not os.path.exists(path):
        raise FileNotFoundError(f'File not found: {path}')
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f'Failed to load image: {path}')
    
    return img

def detect_circles_hough(gray: np.ndarray):
    h, w = gray.shape
    blur = cv2.GaussianBlur(gray, (7, 7), 1.5)

    # 동공 후보
    circles_pupil = cv2.HoughCircles(
        blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=120, param2=18,
        minRadius=15, maxRadius=int(0.18*min(h, w))
    )
    # 홍채 후보
    circles_iris = cv2.HoughCircles(
        blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=150, param2=28,
        minRadius=int(0.22*min(h, w)), maxRadius=int(0.45*min(h, w))
    )

    cx, cy = w // 2, h // 2
    r_in = int(0.14 * min(h, w))
    r_out = int(0.33 * min(h, w))

    if circles_pupil is not None:
        c = circles_pupil[0][0]
        cx, cy, r_in = int(c[0]), int(c[1]), max(int(c[2]), 10)
    
    if circles_iris is not None:
        c = circles_iris[0][0]
        cx = int(0.5 * cx + 0.5 * int(c[0]))
        cy = int(0.5 * cy + 0.5 * int(c[1]))
        r_out = max(int(c[2]), r_in + 15)

    r_out = max(r_out, r_in + 20)

    return (cx, cy), r_in, r_out

def normalize_iris(gray: np.ndarray, center, r_in, r_out, H=64, W=512) -> np.ndarray:
    h, w = gray.shape
    yy, xx = np.indices((H, W))
    theta = 2 * np.pi * xx / W
    r = r_in + (r_out - r_in) * (yy / (H-1))

    X = (center[0] + r * np.cos(theta)).astype(np.int32)
    Y = (center[1] + r * np.sin(theta)).astype(np.int32)

    X = np.clip(X, 0, w-1)
    Y = np.clip(Y, 0, h-1)

    nor = gray[Y, X].astype(np.float32)

    return nor

def log_gabor_id(W: int, f0: float, sigma_f: float) -> np.ndarray:
    freqs = np.fft.fftfreq(W)
    f = np.abs(freqs)
    G = np.zeros_like(f, dtype=np.float32)
    eps = 1e-9
    passband = f > 0
    G[passband] = np.exp(- (np.log(f[passband] / (f0 + eps)) ** 2) / (2 * (np.log(sigma_f) ** 2) + eps))
    G[~passband] = 0.0

    return np.fft.fftshift(G).astype(np.float32)

def encode_iris(nor: np.ndarray, f0s=(0.05, 0.10), sigma_f=1.5) -> np.ndarray:
    H, W = nor.shape
    nor = (nor - np.mean(nor)) / (np.std(nor) + 1e-6)
    codes = []
    for f0 in f0s:
        k = log_gabor_id(W, f0, sigma_f).reshape(1, -1)
        resp_r = convolve2d(nor, k, mode='same', boundary='wrap')
        resp_i = convolve2d(nor, np.roll(k, 1, axis=1), mode='same', boundary='wrap')

        codes.append((resp_r >= 0).astype(np.uint8))
        codes.append((resp_i >= 0).astype(np.uint8))

    code = np.stack(codes, axis=-1)

    return code

def simple_mask(nor: np.ndarray, low=5, high=250, eyelid_rows=5) -> np.ndarray:
    m = (nor > low) & (nor < high)
    m[:eyelid_rows, :] = 0
    m[-eyelid_rows:, :] = 0

    return m.astype(np.uint8)

def save_template(subj: int, eye: str, code: np.ndarray, mask: np.ndarray, meta: dict):
    out = os.path.join(TEMPLATE_DIR, f'S{subj}_{eye}.npz')
    np.savez_compressed(out, code=code, mask=mask, **meta)

    return out

# img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
# center, r_in, r_out = detect_circles_hough(img)
# nor = normalize_iris(img, center, r_in, r_out)

# # cv2.imshow('Normalized Iris', nor.astype(np.uint8))
# # cv2.waitKey(0)

# code = encode_iris(nor)
# # print(code)

# mask = simple_mask(nor)
# meta = dict(center=np.array(center), r_in=r_in, r_out=r_out, H=nor.shape[0], W=nor.shape[1])
# path = save_template(REG_SUBJ, REG_EYE, code, mask, meta)

def register(subject: int, eye: str, shot: str):
    img = load_iris(subject, eye, shot)
    center, r_in, r_out = detect_circles_hough(img)
    nor = normalize_iris(img, center, r_in, r_out)
    code = encode_iris(nor)
    mask = simple_mask(nor)
    meta = dict(center=np.array(center), r_in=r_in, r_out=r_out, H=nor.shape[0], W=nor.shape[1])
    path = save_template(subject, eye, code, mask, meta)
    
    return dict(template_path=path, center=center, r_in=r_in, r_out=r_out, shape=nor.shape)

reg_info = register(REG_SUBJ, REG_EYE, REG_SHOT)
print('Registered:', reg_info)

In [9]:
# 신원 인증

def load_template(subj: int, eye: str):
    path = os.path.join(TEMPLATE_DIR, f'S{subj}_{eye}.npz')
    if not os.path.exists(path):
        raise FileNotFoundError(f'Template not found: {path}')
    
    return np.load(path, allow_pickle=True)

def hamming_distance(codeA: np.ndarray, codeB: np.ndarray, maskA: np.ndarray = None, maskB: np.ndarray = None, max_shift: int = 8) -> float:
    H, W, _ = codeA.shape
    if maskA is None:
        maskA = np.ones((H, W), dtype=np.uint8)
    if maskB is None:
        maskB = np.ones((H, W), dtype=np.uint8)

    best = 1.0
    for s in range(-max_shift, max_shift+1):
        cb = np.roll(codeB, s, axis=1)
        mb = np.roll(maskB, s, axis=1)

        valid = (maskA & mb).astype(bool)
        if valid.sum() == 0:
            continue

        diff = (codeA != cb).any(axis=2)
        hb = diff[valid].mean()
        best = min(best, hb)
    return best

def authenticate(reg_subj: int, reg_eye: str, test_subj: int, test_eye: str, test_shot: str, max_shift: int = MAX_SHIFT) -> dict:
    db = load_template(reg_subj, reg_eye)

    img_t = load_iris(test_subj, test_eye, test_shot)
    center, r_in, r_out = detect_circles_hough(img_t)
    nor_t = normalize_iris(img_t, center, r_in, r_out, H=db['H'], W=db['W'])
    code_t = encode_iris(nor_t)
    mask_t = simple_mask(nor_t)

    hd = hamming_distance(db['code'], code_t, db['mask'], mask_t, max_shift=max_shift)
    return float(hd)

reg_info = register(REG_SUBJ, REG_EYE, REG_SHOT)
print('Registered:', reg_info)

for i in range(1, 6, 1):
    hd_same = authenticate(REG_SUBJ, REG_EYE, REG_SUBJ, REG_EYE, f'aeval{i}.bmp')
    print(hd_same)

print('--- Different Subject ---')

for i in range(1, 6, 1):
    hd_diff = authenticate(REG_SUBJ, REG_EYE, DIFF_SUBJ, REG_EYE, f'chongpkl{i}.bmp')
    print(hd_diff)

Registered: {'template_path': 'templates/S1_left.npz', 'center': (154, 118), 'r_in': 23, 'r_out': 53, 'shape': (64, 512)}
0.0
0.3411820023148148
0.30063302586362817
0.3597005208333333
0.33036747685185186
--- Different Subject ---
0.3665364583333333
0.6374782986111112
0.39963827093506965
0.3519603587962963
0.4441189236111111
